In [2]:
import librosa
import numpy as np
import os
import pandas as pd

In [3]:
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop','reggae','rock']
dataset = []

In [12]:
import os
import librosa
import numpy as np

dataset = []

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()

for genre in genres:
    folder = f"Data/genres_original/{genre}"
    print(f"\n🎵 Processing genre: {genre}")

    for filename in os.listdir(folder):
        if filename.endswith(".wav"):
            file_path = os.path.join(folder, filename)

            try:
                # Load audio
                y, sr = librosa.load(file_path, sr=None, duration=30)

                # Extract features
                mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
                mfccs_mean = np.mean(mfccs, axis=1)
                mfccs_var = np.var(mfccs, axis=1)

                chroma = librosa.feature.chroma_stft(y=y, sr=sr)
                chroma_mean = np.mean(chroma, axis=1)

                spec_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
                spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
                rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
                zcr = np.mean(librosa.feature.zero_crossing_rate(y))

                tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

                # Combine into one feature vector
                features = np.hstack([
                    mfccs_mean, mfccs_var,
                    chroma_mean,
                    spec_centroid, spec_bw, rolloff, zcr, tempo
                ])

                dataset.append([features, genre])
                print(f"✅ Extracted features from: {filename}")

            except Exception as e:
                print(f"❌ Failed on: {filename}, Skipping!")



🎵 Processing genre: blues
✅ Extracted features from: blues.00000.wav
✅ Extracted features from: blues.00001.wav
✅ Extracted features from: blues.00002.wav
✅ Extracted features from: blues.00003.wav
✅ Extracted features from: blues.00004.wav
✅ Extracted features from: blues.00005.wav
✅ Extracted features from: blues.00006.wav
✅ Extracted features from: blues.00007.wav
✅ Extracted features from: blues.00008.wav
✅ Extracted features from: blues.00009.wav
✅ Extracted features from: blues.00010.wav
✅ Extracted features from: blues.00011.wav
✅ Extracted features from: blues.00012.wav
✅ Extracted features from: blues.00013.wav
✅ Extracted features from: blues.00014.wav
✅ Extracted features from: blues.00015.wav
✅ Extracted features from: blues.00016.wav
✅ Extracted features from: blues.00017.wav
✅ Extracted features from: blues.00018.wav
✅ Extracted features from: blues.00019.wav
✅ Extracted features from: blues.00020.wav
✅ Extracted features from: blues.00021.wav
✅ Extracted features from: 

C:\Users\mazin\AppData\Local\Temp\ipykernel_24600\97983256.py:19: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=None, duration=30)
C:\Users\mazin\OneDrive\Desktop\Elevvo Internship\Music-Genre-Classification\.venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


✅ Extracted features from: jazz.00055.wav
✅ Extracted features from: jazz.00056.wav
✅ Extracted features from: jazz.00057.wav
✅ Extracted features from: jazz.00058.wav
✅ Extracted features from: jazz.00059.wav
✅ Extracted features from: jazz.00060.wav
✅ Extracted features from: jazz.00061.wav
✅ Extracted features from: jazz.00062.wav
✅ Extracted features from: jazz.00063.wav
✅ Extracted features from: jazz.00064.wav
✅ Extracted features from: jazz.00065.wav
✅ Extracted features from: jazz.00066.wav
✅ Extracted features from: jazz.00067.wav
✅ Extracted features from: jazz.00068.wav
✅ Extracted features from: jazz.00069.wav
✅ Extracted features from: jazz.00070.wav
✅ Extracted features from: jazz.00071.wav
✅ Extracted features from: jazz.00072.wav
✅ Extracted features from: jazz.00073.wav
✅ Extracted features from: jazz.00074.wav
✅ Extracted features from: jazz.00075.wav
✅ Extracted features from: jazz.00076.wav
✅ Extracted features from: jazz.00077.wav
✅ Extracted features from: jazz.00

In [10]:
# Convert to DataFrame
X = np.array([x[0] for x in dataset])
y = np.array([x[1] for x in dataset])
df = pd.DataFrame(X)
df['genre'] = y
df

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,genre
0,-113.619385,121.553032,-19.151056,42.345768,-6.371168,18.613033,-13.692060,15.339378,-12.283617,10.973775,...,0.295841,0.314634,0.406560,0.384770,1784.416546,2002.657106,3806.418650,0.083066,123.046875,blues
1,-207.581512,123.997147,8.939115,35.870758,2.919970,21.518787,-8.555368,23.355938,-10.101037,11.906445,...,0.384379,0.255474,0.284959,0.334238,1529.871314,2038.612143,3548.986873,0.056044,67.999589,blues
2,-90.776344,140.448608,-29.100559,31.687340,-13.975971,25.741686,-13.644712,11.623112,-11.775920,9.700466,...,0.334242,0.401911,0.384308,0.588190,1552.637786,1747.382028,3041.089944,0.076301,161.499023,blues
3,-199.462006,150.094727,5.649167,26.870144,1.755441,14.236881,-4.828873,9.297849,-0.753142,8.147393,...,0.246386,0.315209,0.316543,0.382855,1070.110059,1596.244204,2185.061787,0.033309,63.024009,blues
4,-160.291855,126.195770,-35.602501,22.151169,-32.487640,10.862022,-23.357162,0.500523,-11.804770,1.203877,...,0.276555,0.324336,0.555298,0.273030,1835.507008,1748.367477,3581.003346,0.101500,135.999178,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,-153.616516,109.894516,-23.106590,59.007462,-7.527995,19.730127,-16.486076,18.083521,-22.215626,15.403847,...,0.294028,0.434824,0.232974,0.193303,2008.537046,2106.349206,4254.124276,0.089267,103.359375,rock
995,-142.442062,116.238441,-32.190319,49.114605,-8.395415,22.872288,-18.311134,20.106487,-22.106909,10.796756,...,0.562719,0.703376,0.363608,0.322890,2006.009248,2068.224879,4147.166589,0.097659,117.453835,rock
996,-125.065109,115.203308,-48.004681,52.843102,-13.491938,21.765408,-12.428562,20.139965,-18.370886,10.119689,...,0.355934,0.527154,0.386691,0.468720,2077.166788,1926.895810,4030.750627,0.121824,129.199219,rock
997,-224.994019,123.669678,-9.775934,56.605724,10.427107,20.634146,-10.257257,15.573306,-8.251973,12.805178,...,0.424520,0.336628,0.263466,0.341012,1398.581574,1817.813570,3014.673437,0.048731,73.828125,rock
